# Synthesizing circuit diagrams from Q# code

In [1]:
import qsharp

The `dump_circuit()` function displays a circuit that contains the gates that have been applied in the simulator up to this point.

In [2]:
%%qsharp

// Prepare a Bell State.
use register = Qubit[2];
H(register[0]);
CNOT(register[0], register[1]);

In [3]:
qsharp.dump_circuit()

q_0    ── H ──── ● ──
q_1    ───────── X ──

If you have the Q# widgets installed, you can display the circuit as an SVG image.

_Run `pip install qsharp-widgets` in the command line to install the Q# widgets._

In [4]:
from qsharp_widgets import Circuit

Circuit(qsharp.dump_circuit())

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"X","isControlled":tr…

You can syntheisize a circuit diagram for any program by calling `qsharp.circuit()` with an entry expression.

In [5]:
%%qsharp

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation GHZSample(n: Int) : Result[] {
    use qs = Qubit[n];

    H(qs[0]);
    ApplyToEach(CNOT(qs[0], _), qs[1...]);

    let results = MeasureEachZ(qs);
    ResetAll(qs);
    return results;
}

In [6]:
Circuit(qsharp.circuit("GHZSample(3)"))

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"X","isControlled":tr…

Circuit diagrams can also be generated for any operation that takes qubits or arrays of qubits.

The diagram will show as many wires as there are input qubit, plus any additional qubits that are allocated within the operation.

When the operation takes an array of qubits (`Qubit[]`), the circuit will show the array as a register of 2 qubits.

In [7]:
%%qsharp

operation PrepareCatState(register : Qubit[]) : Unit {
    H(register[0]);
    ApplyToEach(CNOT(register[0], _), register[1...]);
}

In [8]:
Circuit(qsharp.circuit(operation="PrepareCatState"))

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"X","isControlled":tr…

Circuit synthesis takes into account the currently chosen target, and will perform the same gate decompositions and other transformations that compiling for that target would produce.

Here, we show what the circuit looks like for a random bit generator when the Unrestricted target profile is chosen.

In [9]:
%%qsharp

operation TwoRandomBits() : Result[] {
    let r1 = RandomBit();
    let r2 = RandomBit();
    return [r1, r2];
}

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    MResetZ(q)
}

In [10]:
Circuit(qsharp.circuit("TwoRandomBits()"))

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"Measure","isMeasurem…

If we generate a circuit for the same program, but targeting the Base profile, the resulting circuit avoids reset gates and uses two qubits instead.

In [11]:
qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': []}

In [12]:
%%qsharp

operation TwoRandomBits() : Result[] {
    let r1 = RandomBit();
    let r2 = RandomBit();
    return [r1, r2];
}

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    MResetZ(q)
}

In [13]:
Circuit(qsharp.circuit("TwoRandomBits()"))

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"H","targets":[{"qId"…

Regardless of the target chosen, conditionals that compare `Result` values are not permitted during circuit synthesis. This is because they may introduce nondeterminism such that the circuit will look different depending on measurement outcome. Representing conditionals visually is not supported.

In [14]:
qsharp.init(target_profile=qsharp.TargetProfile.Unrestricted)

Q# initialized with configuration: {'targetProfile': 'unrestricted', 'languageFeatures': []}

In [15]:
%%qsharp

operation ResetIfOne() : Result {
    use q = Qubit();
    H(q);
    let r = M(q);
    if (r == One) {
        Message("result was One, applying X gate");
        X(q);
    } else {
        Message("result was Zero");
    }
    Reset(q);
    return r
}

In [16]:
# Program can be simulated. Differerent shots may produce different results.
qsharp.run("ResetIfOne()", 3)

# The same program cannot be synthesized as a circuit because of the conditional X gate.
qsharp.circuit("ResetIfOne()")

result was One, applying X gate
result was Zero
result was One, applying X gate


QSharpError: Error: Result comparison is unsupported for this backend
Call stack:
    at ResetIfOne in line_0
[31mQsc.Eval.ResultComparisonUnsupported[0m

  [31m×[0m runtime error
[31m  ╰─▶ [0mResult comparison is unsupported for this backend
   ╭─[[36;1;4mline_0[0m:5:1]
 [2m5[0m │     let r = M(q);
 [2m6[0m │     if (r == One) {
   · [35;1m             ─┬─[0m
   ·               [35;1m╰── [35;1mcannot compare to result[0m[0m
 [2m7[0m │         Message("result was One, applying X gate");
   ╰────


Even though we can't synthesize the above program into a circuit, we still have the option of running it in the simulator, and displaying the resulting circuit.

Note that the resulting circuit diagram shows only one of the two branches that could have been taken.

In [17]:
%%qsharp

ResetIfOne()

result was Zero

Zero

In [18]:
Circuit(qsharp.dump_circuit())

Circuit(circuit_json='{"operations":[{"gate":"H","targets":[{"qId":0,"type":0}]},{"gate":"Measure","isMeasurem…